In [34]:
                                    Universidad Politénica Salesiana
Estudiante: Rayner Palta
    
    Diseñe y desarrolle un sistema recopilador que permita obtener las noticias o facebook de los candidatos y almacenar dentro de una base de datos orientados a grafos:
    Webscraping es la técnica de extraer datos contenidos en un formato no estructurado en una página web y llevarlos a una estructura fácil de usar.
    Es por ello, que se desea crear nuevos métodos que permitan la recopilación masiva de informaciones para su posterior estudio y correlación en forma de big data.
    En base a ello, vamos a obtener los datos de lo que esta hablando las noticias de los candidatos dentro del Ecuador y almacenar los datos dentro de una base de datos orientadas a grafo


In [ ]:
from neo4j import GraphDatabase
class Neo4jService(object):
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
        #self._driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "neo4jj"), encrypted=False)
    def close(self):
        self._driver.close()      
    #nodos principales
    def nodo_cabeceraTitulo(self, tx, nombre):
        tx.run("CREATE (:Titulo {nombre: $nombre})", nombre=nombre)
    def nodo_candidatoAsambleista(self, tx, nombre):
        tx.run("CREATE (:Candidato {nombre: $nombre})", nombre=nombre)
    def nodo_noticia(self, tx, nombre):
        tx.run("CREATE (:Noticias {nombre: $nombre})", nombre=nombre)
    def nodo_fechaNoticia(self, tx, nombre):
        tx.run("CREATE (:Fecha {nombre: $nombre})", nombre=nombre)   
    
    #nodos secundarios
    def nodo_contenidoNoticia(self, tx, nombre):
        tx.run("CREATE (:Contenido {nombre: $nombre})", nombre=nombre)   
    def nodo_titulosNoticias(self, tx, nombre):
        tx.run("CREATE (:Titulos {nombre: $nombre})", nombre=nombre)
    def nodo_fechasNoticias(self, tx, nombre):
        tx.run("CREATE (:Fechas {nombre: $nombre})", nombre=nombre)
     
    #relacionaion
    def relacion_noticia(self, tx, nombre_noticias, nombre_noticia):
        tx.run("MATCH (a:Noticias {nombre: $nombre_noticias}) "
               "MATCH (b:Contenido {nombre: $nombre_noticia}) "
               "MERGE (a)-[:Noticias]->(b)",
               nombre_noticias=nombre_noticias, nombre_noticia=nombre_noticia)
    def relacion_titulosCabeceras(self, tx, nombre_titulos, nombre_titulo):
        tx.run("MATCH (a:Titulo {nombre: $nombre_titulos}) "
               "MATCH (b:Titulos {nombre: $nombre_titulo}) "
               "MERGE (a)-[:TituloNoticia]->(b)",
               nombre_titulos=nombre_titulos, nombre_titulo=nombre_titulo)
    def relacion_fechaNoticias(self, tx, tiempo_fechas, nombre_fecha):
        tx.run("MATCH (a:Fecha {nombre: $tiempo_fechas}) "
               "MATCH (b:Fechas {nombre: $nombre_fecha}) "
               "MERGE (a)-[:Fecha]->(b)",
               tiempo_fechas=tiempo_fechas, nombre_fecha=nombre_fecha)

    #relacion de candidato   titulo fecha contenido Noticia
    def relacion_candidato_tituloNoticia(self, tx, candidato, titulo_noticia):
        tx.run("MATCH (a:Candidato {nombre: $candidato}) "
               "MATCH (b:Titulo {nombre: $titulo_noticia}) "
               "MERGE (a)-[:TituloNoticia_Candidato]->(b)",
               candidato=candidato, titulo_noticia=titulo_noticia)
        
    def relacion_candidato_fechaNoticia(self, tx, candidato, fecha_noticia):
        tx.run("MATCH (a:Candidato {nombre: $candidato}) "
               "MATCH (b:Fecha {nombre: $fecha_noticia}) "
               "MERGE (a)-[:FechaNoticia_Candidato]->(b)",
               candidato=candidato, fecha_noticia=fecha_noticia)
    
    def relacion_candidato_contenidoNoticia(self, tx, candidato, noticia_contenido):
        tx.run("MATCH (a:Candidato {nombre: $candidato}) "
               "MATCH (b:Noticias {nombre: $noticia_contenido}) "
               "MERGE (a)-[:Noticias_Candidato]->(b)",
               candidato=candidato, noticia_contenido=noticia_contenido)

In [35]:
import requests
from bs4 import BeautifulSoup
neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'neo4jj')
with neo4j._driver.session() as session:
    session.write_transaction(neo4j.nodo_candidatoAsambleista , "Candidato")
    session.write_transaction(neo4j.nodo_cabeceraTitulo , "Titulos")
    session.write_transaction(neo4j.nodo_fechaNoticia , "Fecha")
    session.write_transaction(neo4j.nodo_noticia , "Noticias")
    print('Nodos creados')
with open("/Users/rayner/Downloads/links.txt","r") as archivo:
    for linea in archivo:
        #print(linea)
        pageGoo= requests.get(linea)
        soupGoo = BeautifulSoup(pageGoo.content,'html.parser')
        resultaFecha = soupGoo.find_all("span",{"class":"r0bn4c rQMQod"})
        resultaTitulo = soupGoo.find_all("div",{"class":"BNeawe vvjwJb AP7Wnd"})
        resultaTexto = soupGoo.find_all("div",{"class":"kCrYT"}) 
        print("-----INICIO SCRAPING GOOGLE------")
        contenidoTextoGoogle=list()
        for resu in resultaTexto:
            contenidoTextoGoogle.append(resu.text)
        auxGo=list(set(contenidoTextoGoogle)) 
        for ll in auxGo:
            session.write_transaction(neo4j.nodo_contenidoNoticia,ll)
            session.write_transaction(neo4j.relacion_noticia,"Noticias",ll)
        tituloGoo=list()
        for i in resultaTitulo:
            tituloGoo.append(i.text) 
        for titulo in tituloGoo:
            session.write_transaction(neo4j.nodo_titulosNoticias,titulo)
            session.write_transaction(neo4j.relacion_titulosCabeceras,"Titulos",titulo)  
            #print(titulo)
        enlaceFecha=list()
        for e2 in resultaFecha:
            enlaceFecha.append(e2.text)
        for fecha in enlaceFecha:
            session.write_transaction(neo4j.nodo_fechasNoticias,fecha)
            session.write_transaction(neo4j.relacion_fechaNoticias,"Fecha",fecha)  
            
session.write_transaction(neo4j.relacion_candidato_tituloNoticia,"Candidato","Titulos")
session.write_transaction(neo4j.relacion_candidato_contenidoNoticia,"Candidato","Noticias")
session.write_transaction(neo4j.relacion_candidato_fechaNoticia,"Candidato", "Fecha")
print("fin del proceso")

Nodos creados
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
-----INICIO SCRAPING GOOGLE------
